# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
data = pd.read_csv('/datasets/data.csv')
data.head(15)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Описание данных

- children - количество детей в семье
- days_employed - трудовой стаж в днях
- dob_yers - возраст клиента в годах
- education - образование клиента
- education_id - идентификатор образования
- family_status - семейное положение
- family_status_id - идентификатор семейного положения
- gender - пол клиента
- income_type - тип занятости
- debt - имел ли задолженность по возврату кредитов
- total_income - доход в месяц
- purpose - цель получения кредита

В каждой строке - данные о потенциальном заемщике. Количество значений в столбцах различаются, соответственно есть пропущенные значения. Столбцы описывают самого заемщика, и его потенциальные расходы (наличие семьи и детей, цель кредита), а также риски невозврата кредита (задолженность возврата по кредиту, тип занятости, трудовой стаж, возраст). Предварительно можно утверждать, что данных достаточно для проверки гипотезы. В некоторых столбцах данные нужно привести к хорошему стилю, убрать отрицательные значения и перевести в понятную систему. Найти пропуски и проанализировать их количество, и как они могут повлиять на итоговый результат.

## Шаг. Предобработка данных

### Обработка пропусков

- Поочередно берем каждый столбец проверяет возможные пропуски и решаем как их заполнить 

Для нашего анализа платежеспособности важен в первую очередь доход в месяц. Трудовой стаж может быть и неофициальным. Для того чтобы дальше работать с данными нужно заполнить столбцы. Для этого можно разделить на группы по типу занятости и добавить в стобцы стаж и доход медианное значение этих групп.

- Сначала приводим значение столбцов стаж и доход к модулю, чтобы избавиться от отрицательных значений


In [5]:
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].abs() 

In [6]:
data.loc[data['days_employed'] >28000, 'days_employed'] = data.loc[data['days_employed'] >28000, 'days_employed'] / 24

- Скорее всего при заполнении столбца со стажем были допущены ошибки, была другая система исчесления (в часах). Поэтому данные свыше 28000 переводим в часы. Число 28000 взято как максимально реалистичный стаж. 

In [7]:
for tipe in data['income_type'].unique():
    median = data.loc[data['income_type'] == tipe, 'days_employed'].median()
    print (tipe, median)
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == tipe), 'days_employed'] = median

сотрудник 1574.2028211070854
пенсионер 15217.221094405468
компаньон 1547.3822226779334
госслужащий 2689.3683533043886
безработный 15267.235531008522
предприниматель 520.8480834953765
студент 578.7515535382181
в декрете 3296.7599620220594


In [8]:
data['days_employed'].isna().sum()

0

- Заполнен столбец стаж медианным значением по типу занятости. Пропуски удалены.

In [9]:
for tipe in data['income_type'].unique():
    median = data.loc[data['income_type'] == tipe, 'total_income'].median()
    print (tipe, median)
    data.loc[(data['total_income'].isna()) & (data['income_type'] == tipe), 'total_income'] = median

сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995


In [10]:
data['income_type'].isna().sum()

0

- Заполнен столбец доход в месяц медианным значением по типу занятости. Пропуски удалены.

- Столбец образование приведем к нижнему регистру.

In [11]:
data['education'] = data['education'].str.lower()

In [12]:
data['children'].unique() 


array([ 1,  0,  3,  2, -1,  4, 20,  5])

- Предположим, что отрицательное значение появилось из-за опечатки при заполнении данных.

In [13]:
data['children'] = data['children'].abs()
data['children'].unique() 

array([ 1,  0,  3,  2,  4, 20,  5])

In [14]:
print("Количество строк с 20 детьми:", data[data['children'] == 20].count()[0])

Количество строк с 20 детьми: 76


- Скорее всего 20 детей это это опечатка,  исправляем на 2

In [15]:
data['children'] = data['children'].replace(20,2)
data['children'].unique() 

array([1, 0, 3, 2, 4, 5])

In [16]:
print("Количество строк с 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])

Количество строк с 'dob_years' 0 - 101


- Большое количество нулевых значений по возрасту клиента разумнее всего заполнить среднем значением от типа занятости. 

In [17]:
for new in data['income_type'].unique():
    mean = data.loc[data['income_type'] == new, 'dob_years'].mean().astype('int')
    print (new, mean)
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == new), 'dob_years'] = mean

сотрудник 39
пенсионер 59
компаньон 39
госслужащий 40
безработный 38
предприниматель 42
студент 22
в декрете 39


In [18]:
print("Количество строк с 'dob_years' 0 -", data[data['dob_years'] == 0].count()[0])

Количество строк с 'dob_years' 0 - 0


- Все пропуски в столбе заполнены

**Вывод**

Все NaN в нужных для дальнейшего анализы гипотез по датафрейму заполнены, явно некорректные значения откорректированы. 

### Замена типа данных

Столбцам 'days_employed' и 'total_income' стоит присвоить целочисленные значения (текущая точность нам не нужна).
Используем функцию 'astype', т.к. столбцы у нас имеют чистый формат 'float64', т.е. перевод в 'int' будет безопасным.

In [19]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

**Вывод**

Теперь все данные представлены в довольно удобном формате, для восприятия и анализа.
Доход в месяц и стаж в днях перевели в 'int', чтобы видеть целочисленные значения.

### Обработка дубликатов

- Сначала проверим датафрейм на дубликаты.

In [20]:
data.duplicated().sum()

71

In [21]:
data[data.duplicated()].head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,15217,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
6312,0,1574,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
7808,0,15217,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7921,0,15217,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7938,0,15217,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,15217,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование


- Выглядят как дубликаты, предположительно задвоение данных. Возможно подача нескольких заявок на кредит...Возможно технические проблемы при загрузке или выгрузке баз данных.
Удаляем дубликаты.

In [22]:
data = data.drop_duplicates() 
data.duplicated().sum()

0

**Вывод**

Проверили датафрейм на дубликаты с помощью метода duplicated(), визуально просмотрели некоторые из найденных дубликатов. Подтвердив необходимость удаления дубликатов.

### Лемматизация

В нашем датафрейме есть столбец 'purpose' с целью получения кредита. Очень много одинаковых целей, выраженных разными словами.
Нам нужно найти все леммы в этом столбце для создания нового столбца 'purpose_id' в котором будут категории целей кредита. 

In [23]:
data['purpose'].value_counts()


свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [24]:
from collections import Counter
text = data['purpose'].unique()
text = ' '.join(text)
lemmas = m.lemmatize(text)
Counter(lemmas)


Counter({'покупка': 10,
         ' ': 96,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

In [25]:
def lemmas_purpose(row):
    purpose = row['purpose']
    purpose_lemmas = m.lemmatize(purpose)
 
    if ('недвижимость' in purpose_lemmas or 'жилье' in purpose_lemmas):
        return 'жилье'
    elif 'автомобиль'in purpose_lemmas:
        return 'автомобиль'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    else:
        return 'цель не определена'
 
data['purpose_id'] = data.apply(lemmas_purpose, axis=1)    
data['purpose_id'].value_counts()

жилье          10811
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_id, dtype: int64

In [26]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


**Вывод**

Метод Лемматизации позволил разбить цель кредита на четыре категории. 
А также стало видно что около 50 % клиентов берут кредит на приобретение недвижимости.

### Категоризация данных

- Для того чтобы было удобнее делать выводы по нашим гипотезам, проведем категоризацию по двум столбцам это количество детей и месячный доход.


По количеству детей ('children'):
- бездетные (нет детей)
- 1-2 ребенка (1,2 ребенка)
- многодетные (3 и более детей)

По уровню дохода ('total_income'):
- минимальный (менее 40000)
- средний (от 40000 до 100000)
- выше среднего (от 100000 до 300000)
- высокий (от 300000 и выше)

In [27]:
def children_group(row):
    if row['children'] < 1:
        return 'бездетные'
    if row['children'] <= 2:
        return '1-2 ребенка'
    return 'многодетные'


In [28]:
def total_groupe(row):
    if row['total_income'] < 40000:
        return 'минимальный'
    if row['total_income'] <= 100000:
        return 'средний'
    if row['total_income'] <= 300000:
        return 'выше среднего'
    return 'высокий'  



In [29]:
data['children_group'] = data.apply(children_group, axis=1)
data['total_groupe'] = data.apply(total_groupe, axis=1)

**Вывод**

Категоризировали датафрейм по количеству детей и уровню ежемесячного дохода. Вывели категории в новые столбы для наглядности и дальнейшего анализа.

## Шаг. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

- Для того чтобы сделать вывод нужно вывести таблицу с данными о детях и наличии просрочек по кредитам

In [30]:
data_pivot = data.pivot_table(index = ['children_group'],  values = 'debt', aggfunc = ['sum', 'count', 'mean'])
data_pivot.columns = ['debt', 'total','%']
data_pivot = data_pivot.sort_values(by= ['total'], ascending = False)
print (data_pivot.head(10))

                debt  total         %
children_group                       
бездетные       1063  14091  0.075438
1-2 ребенка      647   6983  0.092654
многодетные       31    380  0.081579


**Вывод**

Больш всего просрочек наблюдается в группе, где у заемщиков 1-2 ребенка. Самые ответственные плательщики это группа бездетных.

- Можно предположить что у бездетных заемщиков нет обязательств связанных с тратами на детей. Дохода хватает на выполнение всех обязательств, а также эта категория планирует будущие кредиты и следит за кредитной историей.
- При появление детей возникают траты и часто неожиданные, что в некоторых ситуациях у группы с 1-2 детьми вызывают нехватку средств и соответственно просрочки. Либо у этой группы смещается важность в сторону обеспечения детей, а исполнение обязательств перед банком теряет первоочередность.
- Категория многодетные это уже люди которые понимают траты на детей, и при планирование кредита более здраво оценивают свои финансовые возможности. Как следствие количество просрочек у них тоже уменьшается.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

- Чтобы проверить эту гипотезу построим таблицу с данными о семейном положении и просрочек в этих группах.

In [31]:
report = data.groupby('family_status').agg({'debt':'count'})
report['%'] = report ['debt'] /  report ['debt'].sum()
report

,debt,%
family_status,,
Не женат / не замужем,2810,0.130978
в разводе,1195,0.055701
вдовец / вдова,959,0.044700
гражданский брак,4151,0.193484
женат / замужем,12339,0.575138


**Вывод**

Больше всего просрочек наблюдается у группы официально зарегистрировавших отношения, также достаточно большой показатель у людей живущих вместе. Самые дисциплинированные плательщики это одинокие люди (в разводе или вдовец/вдова).  

- Максимальное количество просрочек у людей официально зарегистрировавших отношения. Почти 58% заемщиков из этой категории нарушают свои обязательства перед банком. Возможно это происходит из-за дополнительных трат на семейные нужды. Обустройство с совместной жизни, а также возможно что в семье работает только один заемщик. Также нельзя исключить появление детей, которые увеличивают расходы.
- 19% просрочек наблюдается в категории гражданский брак, скорее всего это та же группа расходов, что и у группы женат/замужем. 
- всего 13% просрочек наблюдаем у категории не женат/ не замужем. Предпологаю что это заемщики необремененные дополнительными расходами, а также планирующие пользоваться кредитами. Эта категория следит за кредитной историей и скорее всего просрочки возникают из-за необезательности некоторых заемщиков либо серьезных форс-мажорных ситуаций.
- категории в разводе 5,5% и вдовец/вдова 4,4% скорее всего люди имеющие какой-то опыт и соответсвенно более ответственно относятся к планированию своей жизни и потенциальных расходов. Возможно эти люди уже имеют какую-то финансовую подушку для решения своих форс-мажорных ситуаций. Поэтому они самые ответственные плательщики 


### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

- Для вывода по этим условиям строим таблицу по группам дохода и просрочек у этих групп.

In [32]:
data_pivot_two = data.pivot_table(index = ['total_groupe'],  values = 'debt', aggfunc = ['sum', 'count', 'mean'])
data_pivot_two.columns = ['debt', 'totall','%']
data_pivot_two = data_pivot_two.sort_values(by= ['totall'], ascending = False)
print (data_pivot_two.head(10))

               debt  totall         %
total_groupe                         
выше среднего  1281   15508  0.082603
средний         344    4338  0.079299
высокий         106    1483  0.071477
минимальный      10     125  0.080000


**Вывод**

Как не странно, но ни высокий ни низкий доход сильно не влияют на просрочке по кредитам. Все четыре категории отличаются между сабой всего лишь на 11 процентов. Причем максимальное количество просрочек у категории с доходом выше среднего.

- Максимальное количество 8,2% у людей с доходам выше среднего может предполагать, что люди имеющие хороший доход не хотят просчитывать и планировать свои расходы. Большие незапланированне траты приводят к тому что им не хватает финансов закрыть вовремя платеж по кредиту. 
- Средний доход предполагает что люди себя чувствуют менее свободно в расходах, и соответственно заранее планирует траты. Поэтому число просрочек в этой группе меньше 7,9%
- Категория заемщиков с высоким доходам имеет минимальный процент 7,1% просрочек. Им это позволяют доходы. А просрочки возникают возможно в связи с большой занятостью или необезательностью заемщика.
- Минимальных уровень дохода может не позволять в случаях незапланированных расходов вовремя оплатить кредит. Из-за чего и возникают просрочки в этой категории.

### Как разные цели кредита влияют на его возврат в срок?

In [33]:
report_two = data.groupby('purpose_id').agg({'debt':'count'})
report_two['%'] = report_two ['debt'] /  report ['debt'].sum()
report_two

,debt,%
purpose_id,,
автомобиль,4306,0.200708
жилье,10811,0.503915
образование,4013,0.187051
свадьба,2324,0.108325


**Вывод**

При помощи двух методов groupby и pivot_table можно наглядно посмотреть по всем категориям просрочки платежей.
Самые ответственные заямщики, те которые берут кредит на свадьбу. У них минимальных процент просрочек. Заемщики берущие кредит на жилье напротив оказались не готовы планировать свои расходы на длительный срок, в связи с чем имеют 50% из них имеют просрочки по платежам.

- Самая большая категория просрочек 50,3% это кредит на жилье. Возможно это связано с длительным сроком кредита, а также большими суммами. В связи с изменяющимся уровнем жизни и понижением реальных доходов, заемщикам не хватает финансов вовремя оплачивать кредит.
-Категория автомобиль имеет 20% просрочек, можно предположить что заемщики не учитывают дополнительные расходы которые появляются в связи с приобретением автомобиля(ТО, бензин, страховки). Из-за этого вырастают ежемесячные расходы, и возникают проблемы с оплатой в срок.
- Кредит на образование имеет 18,7% просрочек. Возможно они связаны с неоправданными ожиданиями после окончания обучения. Более низкий уровень оплаты труда, чем ожидалось. Возможно более молодые заемщики расчитывали на помощь родителей при оплате кредита. 
- Самый минимальный 10,8% просрочек на кредит на свадьбу. Скорее всего это связано с более доступными суммами и небольшим сроком кредита. Заемщиком проще планить небольшой кредит, непродолжительное время.

## Шаг. Общий вывод

Отвечая на поставленный вопрос "Влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок?" можно дать ответ влияет. По проанализированым данным можно сделать вывод что ответственный заемщик это: бездетный, находящийся в разводе либо вдовец/вдова с уровнем дохода более 300000 берущий кредит на свадьбу и чуть менее надежный берущий кредит на автомобиль и образование. Самый рискованный тип заемщика это: заемщик имеющий 1-2 ребенка замужний/женатый, с доходом выше 100000 и менее 300000, берущий кредит на покупку жилья.